In [24]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [25]:
data_folder = '../data/hirise-map-proj-v3_2/'

In [35]:
def extract_metadata():
    images_and_labels = None
    with open(data_folder + 'labels-map-proj_v3_2_train_val_test.txt', 'r') as f:
        images_and_labels = f.readlines()
    images_and_labels = pd.DataFrame(images_and_labels).replace('\n', '', regex=True)[0].str.split(' ', expand=True)
    images_and_labels.rename(columns={0: 'file_name', 1: 'label', 2: 'type'}, inplace=True)
    return images_and_labels

In [36]:
metadata = extract_metadata()

,file_name,label,type
0,ESP_013049_0950_RED-0067.jpg,7,train
1,ESP_013049_0950_RED-0067-fv.jpg,7,train
2,ESP_013049_0950_RED-0067-brt.jpg,7,train
3,ESP_013049_0950_RED-0067-r90.jpg,7,train
4,ESP_013049_0950_RED-0067-r180.jpg,7,train
...,...,...,...
67805,ESP_018707_2205_RED-0041.jpg,0,test
67806,ESP_018707_2205_RED-0062.jpg,0,test
67807,ESP_018707_2205_RED-0105.jpg,0,test
67808,ESP_018707_2205_RED-0058.jpg,0,test
